In [1]:
# !pip install catboost -q

In [2]:
# pip install mlflow -q

In [1]:
# !pip install lightgbm

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction import DictVectorizer

import pickle
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

# from mlflow import MlflowClient
import optuna

In [3]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
EXPERIMENT_NAME = 'housing-price'

In [4]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/azureuser/MLOps_zoomcamp/mlruns/1', creation_time=1691845828281, experiment_id='1', last_update_time=1691845828281, lifecycle_stage='active', name='housing-price', tags={}>

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f'run_id:{run.info.run_id}, rmse:{run.data.metrics["rmse"]}')

run_id:d5e764b7c11a47909f54d6381d2f2718, rmse:513014.6579161583
run_id:aafb1d5434d5498e9cfd01575f050d5c, rmse:515120.72860757157
run_id:ed6dfd34401941dca87f607eaa2478c1, rmse:515166.62504973076
run_id:9bfb782e3c9e4b1d93a0d61f75998d79, rmse:515423.8397249803
run_id:10085e5cbee54c01879c6e30b34209b1, rmse:515982.21948141


In [7]:
run_id = "332da12b29be4a7fb4a05ce3e9e9d5ff"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="housing_price")

Registered model 'housing_price' already exists. Creating a new version of this model...
2023/08/16 13:07:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: housing_price, version 4
Created version '4' of model 'housing_price'.


<ModelVersion: aliases=[], creation_timestamp=1692191279513, current_stage='None', description=None, last_updated_timestamp=1692191279513, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/332da12b29be4a7fb4a05ce3e9e9d5ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [14]:
model_name = "housing_price"
latest_version = client.get_latest_versions(name=model_name)

In [15]:
latest_version[0]

<ModelVersion: aliases=[], creation_timestamp=1691853856691, current_stage='Staging', description='', last_updated_timestamp=1691951162335, name='housing_price', run_id='b3de1b48f0f7480c85468b6fb837bd97', run_link='', source='/home/azureuser/MLOps_zoomcamp/mlruns/1/b3de1b48f0f7480c85468b6fb837bd97/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [17]:
# client.transition_model_version_stage(
#     name=model_name,
#     version=latest_version[0].version,
#     stage="Production",
#     archive_existing_versions=False
# )

In [18]:
client.get_model_version_stages(
    name=model_name,
    version=latest_version[0].version
)

['None', 'Staging', 'Production', 'Archived']

In [19]:
registered_model = client.get_registered_model(
    name=model_name
)

In [20]:
registered_model.latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1691853856691, current_stage='Staging', description='', last_updated_timestamp=1691951162335, name='housing_price', run_id='b3de1b48f0f7480c85468b6fb837bd97', run_link='', source='/home/azureuser/MLOps_zoomcamp/mlruns/1/b3de1b48f0f7480c85468b6fb837bd97/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1691950576764, current_stage='Production', description=None, last_updated_timestamp=1691952104895, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/332da12b29be4a7fb4a05ce3e9e9d5ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>,
 <ModelVersion: aliases=[], creation_timestamp=1692190105402, current_stage='None', description=None, last_updated_timestamp=1692190105402, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', 

In [21]:
for model in registered_model:
    print(model)

('aliases', {})
('creation_timestamp', 1691853852199)
('description', '')
('last_updated_timestamp', 1692190105402)
('latest_versions', [<ModelVersion: aliases=[], creation_timestamp=1691853856691, current_stage='Staging', description='', last_updated_timestamp=1691951162335, name='housing_price', run_id='b3de1b48f0f7480c85468b6fb837bd97', run_link='', source='/home/azureuser/MLOps_zoomcamp/mlruns/1/b3de1b48f0f7480c85468b6fb837bd97/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>, <ModelVersion: aliases=[], creation_timestamp=1691950576764, current_stage='Production', description=None, last_updated_timestamp=1691952104895, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/332da12b29be4a7fb4a05ce3e9e9d5ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>, <ModelVersion: aliases=[], creation_timestamp=1692190105402, current_s

In [8]:
data_path = 'data/Housing_dataset_train.csv'

In [9]:
df = pd.read_csv(data_path)

In [10]:
X = df.drop(columns=['price'], axis=0)
y = df.price

In [11]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)

In [14]:
y_test

10821    8.855421e+06
11736    9.697916e+05
7885     8.030089e+05
4024     2.297041e+06
12107    1.320822e+06
             ...     
4465     3.945397e+06
2411     3.164969e+06
9474     1.384777e+06
10451    3.484817e+06
8291     3.267138e+06
Name: price, Length: 2800, dtype: float64

In [15]:
pd.merge(X_test, y_test, left_index=True, right_index=True, how='inner')

,ID,loc,title,bedroom,bathroom,parking_space,price
10821,9064,Lagos,Mansion,8.0,4.0,6.0,8.855421e+06
11736,6056,Ondo,Apartment,NaN,1.0,3.0,9.697916e+05
7885,1338,Bauchi,Apartment,2.0,NaN,1.0,8.030089e+05
4024,150,NaN,Penthouse,3.0,1.0,2.0,2.297041e+06
12107,9794,Taraba,Townhouse,2.0,1.0,4.0,1.320822e+06
...,...,...,...,...,...,...,...
4465,280,NaN,Townhouse,6.0,2.0,2.0,3.945397e+06
2411,210,Adamawa,Penthouse,7.0,NaN,6.0,3.164969e+06
9474,3561,Osun,Bungalow,3.0,1.0,4.0,1.384777e+06
10451,12429,Rivers,Townhouse,6.0,6.0,NaN,3.484817e+06


## Modelling

In [15]:
state_to_zone = {
    "Abia": "South-East",
    "Adamawa": "North-East",
    "Akwa Ibom": "South-South",
    "Anambra": "South-East",
    "Bauchi": "North-East",
    "Bayelsa": "South-South",
    "Benue": "North-Central",
    "Borno": "North-East",
    "Cross River": "South-South",
    "Delta": "South-South",
    "Ebonyi": "South-East",
    "Edo": "South-South",
    "Ekiti": "South-West",
    "Enugu": "South-East",
    "Gombe": "North-East",
    "Imo": "South-East",
    "Jigawa": "North-West",
    "Kaduna": "North-West",
    "Kano": "North-West",
    "Katsina": "North-West",
    "Kebbi": "North-West",
    "Kogi": "North-Central",
    "Kwara": "North-Central",
    "Lagos": "South-West",
    "Nasarawa": "North-Central",
    "Niger": "North-Central",
    "Ogun": "South-West",
    "Ondo": "South-West",
    "Osun": "South-West",
    "Oyo": "South-West",
    "Plateau": "North-Central",
    "Rivers": "South-South",
    "Sokoto": "North-West",
    "Taraba": "North-East",
    "Yobe": "North-East",
    "Zamfara": "North-West",
}

In [16]:
house_type_ranks = {
    'Cottage': 1,
    'Bungalow': 2,
    'Townhouse': 3,
    'Terrace duplex': 4,
    'Detached duplex': 5,
    'Semi-detached duplex': 6,
    'Flat': 7,
    'Penthouse': 8,
    'Apartment': 9,
    'Mansion': 10
}

In [17]:
def preprocess(data_path):
    data = pd.read_csv(data_path)

    print(data.columns.tolist())
    
    data['zone'] = data['loc'].map(state_to_zone)
    data['title'] = data['title'].map(house_type_ranks)

    category_frequencies = data['loc'].value_counts(normalize=True)
    loc_frequency_mapping = category_frequencies.to_dict()
    data['loc'] = data['loc'].map(loc_frequency_mapping)

    data['rooms'] = data['bathroom'] + data['bedroom']
    data['bathroom_ratio'] = data['bathroom']/(data['bathroom'] + data['bedroom'])

    data['zone'] = data['zone'].astype('category').cat.codes

    print("_____________________________________________________________________________")
    print(data.head())

    X = data.drop(columns=['price'], axis=0)
    y = data.price

    return X, y



In [18]:
X_, y_ = preprocess(data_path)

['ID', 'loc', 'title', 'bedroom', 'bathroom', 'parking_space', 'price']
_____________________________________________________________________________
      ID       loc  title  bedroom  bathroom  parking_space        price  \
0   3583  0.028309    6.0      2.0       2.0            1.0  1149999.565   
1   2748  0.028227    9.0      NaN       2.0            4.0  1672416.689   
2   9261  0.027570    NaN      7.0       5.0            NaN  3364799.814   
3   2224  0.029786    5.0      5.0       2.0            4.0  2410306.756   
4  10300  0.026340    4.0      NaN       5.0            6.0  2600700.898   

   zone  rooms  bathroom_ratio  
0     2    4.0        0.500000  
1     5    NaN             NaN  
2     5   12.0        0.416667  
3     3    7.0        0.285714  
4     0    NaN             NaN  


In [27]:
X_dicts = X_.to_dict(orient='records')

In [28]:
dv = DictVectorizer()


In [29]:
mlflow.lightgbm.autolog(disable=True)

In [35]:
with mlflow.start_run():

    params = {
        'max_depth': 10,
        'n_estimators': 2000,
        'learning_rate': 0.002712819361612371,
        'colsample_bytree': 0.9484547548287134,
        'subsample': 0.8490126211976283
        }

    mlflow.log_params(params)

    fold_pred = []
    splits = 2
    fold = KFold(n_splits=splits)

    for data_index, test_index in fold.split(X_, y_):
        X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
        y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

        model = LGBMRegressor(**params, objective='rmse')
        model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)])
        model_preds = model.predict(X_test)

        rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
        print(f'err: {rmse}')
        fold_pred.append(rmse)

    RMSE = np.mean(fold_pred)

    mlflow.log_param("splits", splits)
    mlflow.log_metric("rmse", RMSE)
    
    with open('models/lgb.bin', 'wb') as f:
        pickle.dump(model, f)

    mlflow.log_artifact(local_path="models/lgb.bin", artifact_path="models_pickle")
    mlflow.lightgbm.log_model(model, artifact_path="models_mlflow")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000912 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1425.058519


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
err: 529324.2298639654
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000752 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1423.011279
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves.

In [41]:
with mlflow.start_run():

    params = {
        'max_depth': 10,
        'n_estimators': 2000,
        'subsample': 0.84,
        'learning_rate': 0.01,
        'n_estimators' : 2000
        }

    mlflow.log_params(params)

    fold_pred_1 = []
    splits = 10
    fold = KFold(n_splits=splits)

    for data_index, test_index in fold.split(X_, y_):
        X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
        y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

        model = CatBoostRegressor(**params)
        model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)], verbose=0)
        model_preds = model.predict(X_test)

        rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
        print(f'err: {rmse}')
        fold_pred_1.append(rmse)

    RMSE = np.mean(fold_pred_1)

    mlflow.log_param("splits", splits)
    mlflow.log_metric("rmse", RMSE)
    
    with open('models/cat.bin', 'wb') as f:
        pickle.dump(model, f)

    mlflow.log_artifact(local_path="models/cat.bin", artifact_path="models_pickle")
    mlflow.catboost.log_model(model, artifact_path="models_mlflow")

err: 466743.0169847345
err: 524069.56855255977
err: 638941.7498937332
err: 470119.679511906
err: 476696.0579938225
err: 588413.1371192657
err: 552106.7342063364
err: 494535.4359102119
err: 513112.49038561864
err: 633301.7921100028


## Model Registry

In [9]:
client.list_experiments()

AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

# Hyperparameter Tuning

In [ ]:
import optuna

In [ ]:
# def objective(trial):

#     max_depth = trial.suggest_int('rf_max_depth', 2, 32)
#     n_estimators = trial.suggest_int('n_estimators', 100, 4000)
#     learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
#     subsample = trial.suggest_float('subsample', 0, 1)

#     params = {
#         'max_depth':max_depth,
#         'colsample_bytree': colsample_bytree,
#         'learning_rate': learning_rate,
#         'n_estimators': n_estimators,
#         'subsample': subsample,
#     }

#     X_data, X_val, y_data, y_val = data_test_split(X, y, random_state=RANDOM_STATE)

#     LGB = CatBoostRegressor(**params)
#     LGB.fit(X_data, y_data)
#     y_pred = LGB.predict(X_val)

#     error = mean_squared_error(y_val, y_pred, squared=False)

#     return error  # An objective value linked with the Trial object.

#  # Invoke optimization of the objective function.

In [8]:
# with mlflow.start_run():

#     params = {
#         'max_depth': 10,
#         'n_estimators': 2000,
#         'learning_rate': 0.002712819361612371,
#         'colsample_bytree': 0.9484547548287134,
#         'subsample': 0.8490126211976283
#         }

#     mlflow.log_params(params)

#     fold_pred = []
#     splits = 2
#     fold = KFold(n_splits=splits)

#     for data_index, test_index in fold.split(X_, y_):
#         X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
#         y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

#         model = LGBMRegressor(**params, objective='rmse')
#         model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)])
#         model_preds = model.predict(X_test)

#         rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
#         print(f'err: {rmse}')
#         fold_pred.append(rmse)

#     RMSE = np.mean(fold_pred)

#     mlflow.log_param("splits", splits)
#     mlflow.log_metric("rmse", RMSE)
    
#     with open('models/lgb.bin', 'wb') as f:
#         pickle.dump(model, f)

#     # mlflow.log_artifact(local_path="models/lgb.bin", artifact_path="models_pickle")
#     mlflow.lightgbm.log_model(model, artifact_path="models_mlflow")

In [10]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "lgboost")
        mlflow.log_params(params)
        
        fold_pred = []
        splits = 5
        fold = KFold(n_splits=splits)

        for data_index, test_index in fold.split(X_, y_):
            X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
            y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

            model = LGBMRegressor(**params, objective='rmse')
            model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)])
            model_preds = model.predict(X_test)

            rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
            print(f'err: {rmse}')
            fold_pred.append(rmse)

        RMSE = np.mean(fold_pred)

        mlflow.log_param("splits", splits)
        mlflow.log_metric("rmse", RMSE)

    return RMSE

In [11]:
max_depth = trial.suggest_int('rf_max_depth', 2, 16)
n_estimators = trial.suggest_int('n_estimators', 100, 4000)
learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
subsample = trial.suggest_float('subsample', 0, 1)

params = {
    'max_depth':max_depth,
    'colsample_bytree': colsample_bytree,
    'learning_rate': learning_rate,
    'n_estimators': n_estimators,
    'subsample': subsample,
}

NameError: name 'trial' is not defined

In [19]:
def objective(trial):

    with mlflow.start_run():

        max_depth = trial.suggest_int('rf_max_depth', 2, 16)
        n_estimators = trial.suggest_int('n_estimators', 100, 4000)
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
        subsample = trial.suggest_float('subsample', 0, 1)

        params = {
            'max_depth':max_depth,
            'colsample_bytree': colsample_bytree,
            'learning_rate': learning_rate,
            'n_estimators': n_estimators,
            'subsample': subsample,
        }

        mlflow.log_params(params)

        fold_pred = []
        splits = 5
        fold = KFold(n_splits=splits)

        for data_index, test_index in fold.split(X_, y_):
            X_data, X_test = X_.iloc[data_index], X_.iloc[test_index]
            y_data, y_test = np.sqrt(y_.iloc[data_index]), y_.iloc[test_index]

            model = LGBMRegressor(**params, objective='rmse')
            model.fit(X_data, y_data, eval_set=[(X_data, y_data), (X_test, y_test)])
            model_preds = model.predict(X_test)

            rmse = mean_squared_error(y_test, np.square(model_preds), squared=False)
            print(f'err: {rmse}')
            fold_pred.append(rmse)

        RMSE = np.mean(fold_pred)

        mlflow.log_param("splits", splits)
        mlflow.log_metric("rmse", RMSE)
        
        with open('models/lgb.bin', 'wb') as f:
            pickle.dump(model, f)

        mlflow.log_artifact(local_path="models/lgb.bin", artifact_path="models_pickle")
        mlflow.lightgbm.log_model(model, artifact_path="models_mlflow")

    return RMSE  # An objective value linked with the Trial object.

In [20]:
study = optuna.create_study(direction='minimize')  # Create a new study.
study.optimize(objective, n_trials=10)

[I 2023-08-16 13:17:39,759] A new study created in memory with name: no-name-0cd7bac7-82fa-49fc-a68c-e66a24cf5edb


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008784 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 11200, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1423.804020
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
err: 557441.3544420773
[LightGBM] [Warning] Accuracy may be bad since 

/home/azureuser/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[I 2023-08-16 13:17:50,700] Trial 0 finished with value: 583052.301024107 and parameters: {'rf_max_depth': 11, 'n_estimators': 596, 'learning_rate': 0.004635363527228308, 'colsample_bytree': 0.9174054005728952, 'subsample': 0.7352990344774227}. Best is trial 0 with value: 583052.301024107.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000671 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 11200, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 1423.804020
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be ba

[I 2023-08-16 13:18:06,778] Trial 1 finished with value: 540141.3350229821 and parameters: {'rf_max_depth': 10, 'n_estimators': 2016, 'learning_rate': 0.05282597067670016, 'colsample_bytree': 0.1706591333401979, 'subsample': 0.5737312462462926}. Best is trial 1 with value: 540141.3350229821.


...

[I 2023-08-16 13:20:20,172] Trial 7 finished with value: 1072370.9909723853 and parameters: {'rf_max_depth': 15, 'n_estimators': 1103, 'learning_rate': 2.4599668227204557e-05, 'colsample_bytree': 0.9490348405572879, 'subsample': 0.9535086655634702}. Best is trial 6 with value: 533861.2215643647.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000785 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 11200, number of used features: 9
[LightGBM] [Info] Start training from score 1423.804020
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2023-08-16 13:20:26,730] Trial 8 finished with value: 1049363.0640975821 and parameters: {'rf_max_depth': 4, 'n_estimators': 728, 'learning_rate': 0.0002581572525408428, 'colsample_bytree': 0.2615229533869723, 'subsample': 0.30568046236986723}. Best is trial 6 with value: 533861.2215643647.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 11200, number of used features: 9
[LightGBM] [Info] Start training from score 1423.804020
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2023-08-16 13:20:31,468] Trial 9 finished with value: 911095.5193719994 and parameters: {'rf_max_depth': 3, 'n_estimators': 526, 'learning_rate': 0.0034753449890002094, 'colsample_bytree': 0.02869997475759467, 'subsample': 0.05552198693788446}. Best is trial 6 with value: 533861.2215643647.


In [22]:
trial = study.best_trial

In [23]:
trial.value

533861.2215643647

In [24]:
trial.params

{'rf_max_depth': 6,
 'n_estimators': 3914,
 'learning_rate': 0.014025985859548822,
 'colsample_bytree': 0.21383255211038665,
 'subsample': 0.3887821186692507}

In [43]:
runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=10,
        # order_by=["metrics.rmse"]
    )

In [49]:
best_run = float('inf')
run_id = None
for run in runs:
    print(run.info.run_id, run.data.metrics["rmse"])
    if run.data.metrics["rmse"] < best_run:
        best_run = run.data.metrics["rmse"]
        run_id = run.info.run_id
best_run, run_id

d1cecb4a90e549b0b464757cdc6d8fb9 911095.5193719994
f32f96a98ed84446be15a4a93017c66f 1049363.0640975821
48096c6d543e475d91523345274140c4 1072370.9909723853
58dd1fdb99ce4f26b9b60fe02129a74a 533861.2215643647
c9ed38cab16f465aa47d590fe1abb0b9 960717.9550722272
5e8b8ef4976643f7952b1697f212d68a 1044373.7589869536
976b4f32b5fa4f77aa62d9f6862af98d 1071792.6109700385
5b63e490a3a04b6b88f663d6ecaab255 1040556.517829923
2d83141fdc604498bc9337bda0fd7083 540141.3350229821
3d1a37e45e7b48798cdc1a923c2021ae 583052.301024107


(533861.2215643647, '58dd1fdb99ce4f26b9b60fe02129a74a')

In [53]:
model_registry_name = "housing_price"
version_name = 1

In [61]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name=model_registry_name
)

Registered model 'housing_price' already exists. Creating a new version of this model...
2023/08/16 14:00:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: housing_price, version 6
Created version '6' of model 'housing_price'.


<ModelVersion: aliases=[], creation_timestamp=1692194442478, current_stage='None', description=None, last_updated_timestamp=1692194442478, name='housing_price', run_id='58dd1fdb99ce4f26b9b60fe02129a74a', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/58dd1fdb99ce4f26b9b60fe02129a74a/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [63]:
client.search_model_versions()

[<ModelVersion: aliases=[], creation_timestamp=1692194442478, current_stage='None', description=None, last_updated_timestamp=1692194442478, name='housing_price', run_id='58dd1fdb99ce4f26b9b60fe02129a74a', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/58dd1fdb99ce4f26b9b60fe02129a74a/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=6>,
 <ModelVersion: aliases=[], creation_timestamp=1692194365444, current_stage='None', description=None, last_updated_timestamp=1692194365444, name='housing_price', run_id='58dd1fdb99ce4f26b9b60fe02129a74a', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/58dd1fdb99ce4f26b9b60fe02129a74a/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=5>,
 <ModelVersion: aliases=[], creation_timestamp=1691853856691, current_stage='Staging', description='', last_updated_timestamp=1692193821967, name='housing_price', run_id='b3de1b48f0f7480c85468b6fb837bd97', run_link='

In [72]:
latest_version = client.get_latest_versions(name=model_registry_name, stages=['None'])[0].version

In [73]:
client.transition_model_version_stage(
            name=model_registry_name,
            version=latest_version,
            stage="Staging",
            archive_existing_versions=False,
        )

<ModelVersion: aliases=[], creation_timestamp=1692194442478, current_stage='Staging', description=None, last_updated_timestamp=1692195196465, name='housing_price', run_id='58dd1fdb99ce4f26b9b60fe02129a74a', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/58dd1fdb99ce4f26b9b60fe02129a74a/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [41]:
housing_details = {
    "ID": 343,
    "loc": "lagos",
    "title": "Mansion",
    "bedroom": 2.0,
    "bathroom": 1.0,
    "parking_space" : 2.0
}


In [42]:
x = pd.DataFrame([housing_details])

In [43]:
from prepare_features import prepare

In [44]:
prepare(x)

,ID,loc,title,bedroom,bathroom,parking_space,zone,rooms,bathroom_ratio
0,343,1.0,10,2.0,1.0,2.0,-1,3.0,0.333333


In [45]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
EXPERIMENT_NAME = 'housing-price'

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

model_registry_name = "housing_price"

client = MlflowClient()
prod_model = client.get_latest_versions(model_registry_name, stages=["Production"])[0]

In [46]:
prod_model

<ModelVersion: aliases=[], creation_timestamp=1691950576764, current_stage='Production', description=None, last_updated_timestamp=1691952104895, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/332da12b29be4a7fb4a05ce3e9e9d5ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [47]:
prod_model.version

2

In [48]:
run_id = prod_model.run_id
run_id

'332da12b29be4a7fb4a05ce3e9e9d5ff'

In [49]:
logged_model = f'runs:/{run_id}/model'
logged_model

'runs:/332da12b29be4a7fb4a05ce3e9e9d5ff/model'

In [50]:
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [58]:
float(loaded_model.predict(prepare(x))[0])

1246.8361280347074

In [21]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.lightgbm
  run_id: 332da12b29be4a7fb4a05ce3e9e9d5ff

In [ ]:
model_name = "housing_price"
latest_version = client.get_latest_versions(name=model_name)


latest_version[0].version

client.transition_model_version_stage(
    name=model_name,
    version=latest_version[0].version,
    stage="Staging",
    archive_existing_versions=False
)

In [93]:
client.search_model_versions()

[<ModelVersion: aliases=[], creation_timestamp=1691952653200, current_stage='None', description=None, last_updated_timestamp=1691952653200, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/332da12b29be4a7fb4a05ce3e9e9d5ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>,
 <ModelVersion: aliases=[], creation_timestamp=1691950576764, current_stage='Production', description=None, last_updated_timestamp=1691952104895, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/332da12b29be4a7fb4a05ce3e9e9d5ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>,
 <ModelVersion: aliases=[], creation_timestamp=1691853856691, current_stage='Staging', description='', last_updated_timestamp=1691951162335, name='housing_price', run_id='b3de1b48f0f7480c85468b6fb837bd97', run_li

In [66]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

In [75]:
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)[0]

In [76]:
for run in runs:
    print(f'run_id:{run.info.run_id}, rmse:{run.data.metrics["rmse"]}')

AttributeError: 'tuple' object has no attribute 'info'

In [81]:
best_run_id = runs.info.run_id
best_run_id

'332da12b29be4a7fb4a05ce3e9e9d5ff'

In [82]:
best_run_metric = runs.data.metrics["rmse"]
best_run_metric

530561.7354094362

In [83]:
prod_model = client.get_latest_versions(model_registry_name, stages=["Production"])[0]
prod_model_run_id = prod_model.run_id
prod_model_run_id

'332da12b29be4a7fb4a05ce3e9e9d5ff'

In [84]:
prod_model

<ModelVersion: aliases=[], creation_timestamp=1691950576764, current_stage='Production', description=None, last_updated_timestamp=1691952104895, name='housing_price', run_id='332da12b29be4a7fb4a05ce3e9e9d5ff', run_link=None, source='/home/azureuser/MLOps_zoomcamp/mlruns/1/332da12b29be4a7fb4a05ce3e9e9d5ff/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [89]:
client.get_run(run_id=prod_model_run_id).data.metrics["rmse"]

530561.7354094362